### Get Image and Catalog

In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os,time
import glob,sys

from astropy.io import fits
from astropy.visualization import ZScaleInterval

from RetrieveSource import *
from EstimateBackground import *
from TractorTools import *
from Heasarc import *
from PSF import *

In [2]:
from GetInfoFromDrive import get_file

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=465987015783-enajp1p9qkfhldmamsaj7vgqmtofsldc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [9]:
id = 40415
segment = 1
extension = 1
filter ='um2'
uvfilter = 'UVM2'

HDR,CAT = get_file(id,segment,filter,extension,'smc')

Found Catalog
Found Image


In [16]:
# Step 1: Get Zaritsky Coordinates and Catalog 
#uvfilter = h.header['FILTER']
shape = np.shape(HDR[0].data)

print('---Getting Catalog---')

meta = get_meta().with_hdu(HDR[0],
                           optical_catalog=CAT,
                           xdim=[int(shape[1]/2)-50,int(shape[1]/2)+50],
                           ydim=[int(shape[0]/2)-50,int(shape[0]/2)+50])

print('---Getting Background---')
bkgd = BkgdEstimator(meta,n_pix = [20,20])


print('---Getting PSF---')
pix_scale = np.abs(meta.cdelt)*3600.

psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                     uvfilter = uvfilter, width = 23).psf


print('---Running Tractor---')

fwhm = 2.5
sigma =  fwhm/2.355
# Restrict by a factor of 4.
sigma = sigma/4.
        
TractorObject = PhotometryTools(meta,
                    psf_filename = psf_object,
                    fits_table_sname = f"temp_tractor.fits", 
                    background = bkgd,
                    fit_positions = sigma,
                    threshold = 1.5)

---Getting Catalog---


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55466.000949 from DATE-OBS.
Set MJD-END to 55466.003194 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55466.000949 from DATE-OBS.
Set MJD-END to 55466.003194 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55466.000949 from DATE-OBS.
Set MJD-END to 55466.003194 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55466.000949 from DATE-OBS.
Set MJD-END to 55466.003194 from DATE-END'. [astropy.wcs.wcs]


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
---Getting Backgroun

23
Sum Before Normalization  616.9218729335043
Sum After Normalization  0.9999999999999998
---Running Tractor---
dlnp 181530.06074293764
dlnp 62.13163336143771
dlnp 4.414528791279281
dlnp 0.2108856462523363
dlnp 0.012780556796769815
dlnp 0.001627973103950353
dlnp 0.0002503524256098899


In [ ]:

#         # Step 5: Run Heasarc 
#         t3 = time.time()
#         Magnitudes[uvfilter] = HeasarcRoutines(f"{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.fits",uvfilter)# Should change 1_1 to include all segments!
#         print("Time to Run Heasarc: ",(time.time()-t3)/60)
    
#         # Step 6: Format CSV
#         t4 = time.time()    
    
#         # Get csv created in Heasarc 
#         df = pd.read_csv(f'{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.csv')# Should change 1_1 to include all segments!

#         labels = [uvfilter + '_'+ key for key in df.keys()]
#         d = {}
#         for old,new in zip(df.keys(),labels):
#             d[old] = new
#         df = df.rename(columns=d)

#         print(f'Joining Optical catalog with {uvfilter}')
#         by_filter[uvfilter] = pd.merge(meta[uvfilter].catalog,df,left_on='KEY',right_on=f'{uvfilter}_KEY',how="inner")
#         print("Time to Create CSV by Filter: ",(time.time()-t4)/60)
        
    
#         by_filter[uvfilter] = by_filter[uvfilter][by_filter[uvfilter][uvfilter+'_MAG_ERR'] < 0.35]

#         return TractorObjects